<a href="https://colab.research.google.com/github/ArinaWhyNot/myfirstrep/blob/main/Lecture04_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("/content/mlbootcamp5_train.csv", sep=';')
df.head()
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [15]:
# A lot of code here
#создание метода к-ближних соседей без sklearn
from scipy.spatial import distance
from collections import Counter

class KNNClassifier:
    def __init__(self, k=5, metric='euclidean', weights='uniform'):
        self.k = k
        self.metric = metric
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X.values if isinstance(X, pd.DataFrame) else X
        self.y_train = y.values if isinstance(y, pd.Series) else y

    def predict(self, X):
        X_test = X.values if isinstance(X, pd.DataFrame) else X
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # Вычисляем расстояния
        if self.metric == 'euclidean':
            dists = distance.cdist([x], self.X_train, 'euclidean')[0]
        elif self.metric == 'manhattan':
            dists = distance.cdist([x], self.X_train, 'cityblock')[0]
        else:
            raise ValueError("Метрика должна быть 'euclidean' или 'manhattan'")

        # Получаем k ближайших соседей
        k_indices = np.argsort(dists)[:self.k]
        k_nearest_labels = self.y_train[k_indices]

        # Взвешенное или обычное голосование
        if self.weights == 'uniform':
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        elif self.weights == 'distance':
            k_dists = dists[k_indices]
            weights = 1 / (k_dists + 1e-10)  # Добавляем малое число, чтобы избежать деления на 0
            weighted_votes = {}
            for label, weight in zip(k_nearest_labels, weights):
                if label in weighted_votes:
                    weighted_votes[label] += weight
                else:
                    weighted_votes[label] = weight
            return max(weighted_votes.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Веса должны быть 'uniform' или 'distance'")

In [16]:
#кроссвалидация
def cross_val_score(model, X, y, cv=5, scoring='accuracy'):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        if scoring == 'accuracy':
            score = np.mean(y_pred == y_val)
        elif scoring == 'f1':
            from sklearn.metrics import f1_score
            score = f1_score(y_val, y_pred)
        else:
            raise ValueError("Метрика должна быть 'accuracy' или 'f1'")

        scores.append(score)

    return np.mean(scores)

def find_best_k(X, y, k_values=range(1, 21), cv=5):
    best_k = None
    best_score = -1

    for k in k_values:
        model = KNNClassifier(k=k)
        score = cross_val_score(model, X, y, cv=cv)
        print(f"k={k}, Score={score:.4f}")

        if score > best_score:
            best_score = score
            best_k = k

    print(f"\nЛучшее k: {best_k} с score={best_score:.4f}")
    return best_k

In [13]:
#Реализация метода
# Выбор признаков и целевой переменной
X = df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
          'cholesterol', 'gluc', 'smoke', 'alco', 'active']]
y = df['cardio']

# Нормализация данных (важно для KNN)
X_normalized = (X - X.mean()) / X.std()

# Поиск лучшего k
best_k = find_best_k(X_normalized, y, k_values=range(1, 31), cv=5)

# Обучение финальной модели
final_model = KNNClassifier(k=best_k)
final_model.fit(X_normalized, y)

# Предсказание для новых данных (пример)
new_df = pd.DataFrame([[18393, 2, 168, 62.0, 110, 80, 1, 1, 0, 0, 1]],
                        columns=X.columns)
new_data_normalized = (new_df - X.mean()) / X.std()
prediction = final_model.predict(new_data_normalized)
print(f"Предсказание: {prediction[0]}")

k=1, Score=0.6126
k=2, Score=0.6126
k=3, Score=0.6403
k=4, Score=0.6404
k=5, Score=0.6503
k=6, Score=0.6530
k=7, Score=0.6565
k=8, Score=0.6591
k=9, Score=0.6598
k=10, Score=0.6626
k=11, Score=0.6644
k=12, Score=0.6668
k=13, Score=0.6650
k=14, Score=0.6670
k=15, Score=0.6657
k=16, Score=0.6687
k=17, Score=0.6663
k=18, Score=0.6685
k=19, Score=0.6668
k=20, Score=0.6685
k=21, Score=0.6671
k=22, Score=0.6694
k=23, Score=0.6669
k=24, Score=0.6691
k=25, Score=0.6680
k=26, Score=0.6699
k=27, Score=0.6689
k=28, Score=0.6706
k=29, Score=0.6683
k=30, Score=0.6695

Лучшее k: 28 с score=0.6706
Предсказание: 0


K-ближайших соседей классификатор достиг точности 67.06% при k=28 на кросс-валидации, а для тестового примера предсказал класс 0 (отсутствие сердечно-сосудистого заболевания).

**Комментарии:** Ваши комментарии здесь.
Принцип к-ближних соседей: для нового объекта мы ищем k самых похожих объектов в обучающей выборке и смотрим, к каким классам они относятся. Если большинство "соседей" принадлежат к классу 1, то и наш новый объект скорее всего относится к классу 1.

гиперпараметры:
1) k — количество соседей, которые участвуют в голосовании
2) Метрика расстояния — "похожесть" объектов. Добавили две самые популярные: евклидову и манхэттенскую
3) Веса — можно либо считать всех соседей равнозначными, либо давать больший вес тем, кто ближе к объекту

Чтобы выбрать оптимальное значение k, мы использовали кросс-валидацию.

Еще мы нормализовали данные (привели все признаки к одинаковому масштабу), потому что этот метод очень чувствителен к разным масштабам признаков.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [22]:
# Your code here
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

X = df[['age', 'weight', 'height', 'ap_lo', 'ap_hi']]
y = df['cardio']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

#инициализация моделей
k = KNeighborsClassifier(n_neighbors=28)
nb = GaussianNB()
tree = DecisionTreeClassifier(max_depth=5, random_state=42)

#обучение трех моделей
k.fit(X_train, y_train)
nb.fit(X_train, y_train)
tree.fit(X_train, y_train)

#предсказания
y_pred_k = k.predict(X_test)
y_proba_k = k.predict_proba(X_test)[:, 1]

y_pred_nb = nb.predict(X_test)
y_proba_nb = nb.predict_proba(X_test)[:, 1]

y_pred_tree = tree.predict(X_test)
y_proba_tree = tree.predict_proba(X_test)[:, 1]

#метрики
metrics = {
    'Accuracy': [accuracy_score(y_test, y_pred_k), accuracy_score(y_test, y_pred_nb), accuracy_score(y_test, y_pred_tree)],
    'F1': [f1_score(y_test, y_pred_k), f1_score(y_test, y_pred_nb), f1_score(y_test, y_pred_tree)],
    'ROC AUC': [roc_auc_score(y_test, y_proba_k), roc_auc_score(y_test, y_proba_nb), roc_auc_score(y_test, y_proba_tree)],
    'Log Loss': [log_loss(y_test, y_proba_k), log_loss(y_test, y_proba_nb), log_loss(y_test, y_proba_tree)]
}

results = pd.DataFrame(metrics, index=['k-ближ соседи', 'Naive Bayes', 'Decision Tree'])
print(results)

               Accuracy        F1   ROC AUC  Log Loss
k-ближ соседи  0.683476  0.662777  0.742106  0.603667
Naive Bayes    0.564429  0.322996  0.680523  0.707415
Decision Tree  0.723952  0.713573  0.787783  0.554582


**Комментарии:** Ваши комментарии здесь.
Decision Tree — лучший классификатор по всем метрикам для данного набора признаков. Однако kNN тоже показывает достойные результаты и может быть улучшен. Naive Bayes требует серьезной доработки или замены.